## Tutorial notebook
This notebook is mostly following a tutorial over the paper 

In [28]:
!pip install -r ../requirements.txt

In [29]:
import os
import re

import json
import numpy as np
from typing import Tuple


from langchain.llms import OpenAI

from langchain_experimental.tot.base import ToTChain
from langchain_experimental.tot.checker import ToTChecker
from langchain_experimental.tot.thought import ThoughtValidity

##Load API key ------------------
f = open('../apikeys/api.txt', 'r')
key = f.readline()
os.environ['OPENAI_API_KEY'] = key

In [30]:
class MyChecker(ToTChecker):
    def evaluate(self,
        problem_description: str,
        thoughts: Tuple[str, ...] = ()) -> ThoughtValidity:
        
        last_thought = thoughts[-1]
        clean_solution = last_thought.replace(" ", "").replace('"', "")
        regex_solution = clean_solution.replace("*", ".").replace("|", "\\|")
        if sudoku_solution in clean_solution:
            return ThoughtValidity.VALID_FINAL
        elif re.search(regex_solution, sudoku_solution):
            return ThoughtValidity.VALID_INTERMEDIATE
        else:
            return ThoughtValidity.INVALID


In [31]:
sudoku_puzzle = "3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1"
sudoku_solution = "3,4,1,2|1,2,3,4|2,1,4,3|4,3,2,1"

problem_description = f"""
    {sudoku_puzzle}

    - This is a 4x4 Sudoku puzzle.
    - The * represents a cell to be filled.
    - The | character separates rows.
    - At each step, replace one or more * with digits 1-4.
    - There must be no duplicate digits in any row, column or 2x2 subgrid.
    - Keep the known digits from previous valid thoughts in place.
    - Each thought can be a partial or the final solution.
    """.strip()

In [35]:
llm = OpenAI(temperature=0.1, max_tokens=512, model="text-davinci-003")
checker = MyChecker()

tot_chain = ToTChain(llm=llm, checker=MyChecker(), k=30, c= 2, verbose=True, verbose_llm=False)
solution = tot_chain.run(problem_description=problem_description)



> Entering new ToTChain chain...
Starting the ToT solve procedure.


/usr/local/lib/python3.9/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,2,*,1
Thought: 3,*,*,2|1,4,3,*|*,1,*,3|4,2,*,1
Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,2,*,1
Thought: 3,*,*,2|1,4,3,*|*,1,*,3|4,2,*,1
Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,2,*,1
Thought: 3,*,*,2|1,4,3,*|*,1,*,3|4,2,*,1
Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,2,*,1
Thought: 3,*,*,2|1,4,3,*|*,1,*,3|4,2,*,1
Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,2,*,1
Thought: 3,*,*,2|1,4,3,*|*,1,*,3|4,2,*,1
Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,2,*,1
Thought: 3,*,*,2|1,4,3,*|*,1,*,3|4,2,*,1
Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,2,*,1
Thought: 3,*,*,2|1,4,3,*|*,1,*,3|4,2,*,1
Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,2,*,1
Thought: 3,*,*,2|1,4,3,*|*,1,*,3|4,2,*,1
Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,2,*,1
Thought: 3,*,*,2|1,4,3,*|*,1,*,3|4,2,*,1
Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,2,*,1
Thought: 3,*,*,2|1,4,3,*|*,1,*,3|4,2,*,1
Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,2,*,1
Thought: 3,*,*,2|1,4,3,*|*,1,*,3|4,2,*,1
Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,2,*,1
Thought: 3,*,*,2|1,4,3,*|*,1,*,3|4,2,*,1
Thought: 3,*,*,2

In [36]:
solution

'No solution found'